## Interface Protocol
Server Address and Port
- Address: 127.0.0.1 (localhost loopback)
- Port: 12345

In [ ]:
import socket
import numpy as np
import cv2
import matplotlib.pyplot as plt

def receive_and_show_image():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    try:
        # connect to the server
        client_socket.connect(('127.0.0.1', 12345))
        
        # send the image request
        client_socket.sendall(b'lastimage\n')
        
        # revice the headers
        header = b''
        while not header.endswith(b'\n'):
            chunk = client_socket.recv(1)
            if not chunk: 
                print("no response from server")
                return
            header += chunk
        
        if header.startswith(b'IMAGE:'):
            # get the image data size
            image_size = int(header[6:-1])
            
            if image_size <= 0:
                print("Invalid image data size.")
                return
            
            # recive image data
            image_data = b''
            while len(image_data) < image_size:
                remaining = image_size - len(image_data)
                packet = client_socket.recv(remaining)
                if not packet:
                    print("Reception fof image data interrupted")
                    return
                image_data += packet
            
            if len(image_data) != image_size:
                print("The received image data is incomplete")
                return
            
            # use opencv read image data form menmory
            image_array = np.frombuffer(image_data, dtype=np.uint8)
            img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
            
            if img is not None:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
                # show image
                plt.imshow(img_rgb)
                plt.axis('off')
                plt.show()
            else:
                print("Can not decode image data")
        else:
            print("Error:", header.decode().strip())
    except Exception as e:
        print(f"Error interrupt: {e}")
    finally:
        # make sure socket is closed
        client_socket.close()

receive_and_show_image()

### Ask image data
Must after capture the screenshot from target, then can receive image.
- Data header: "IMAGE:" check the image header.
- Image size: Check the image sieze after the image header.
- Image data: decode image data.
- Show image on notebook.

In [ ]:
import socket
import numpy as np
import cv2
import matplotlib.pyplot as plt

def receive_and_show_image():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    try:
        client_socket.connect(('127.0.0.1', 12345))

        client_socket.sendall(b'lastimage\n')

        header = b''
        while not header.endswith(b'\n'):
            chunk = client_socket.recv(1)
            if not chunk:  
                print("no response from server")
                return
            header += chunk
        
        if header.startswith(b'IMAGE:'):
            image_size = int(header[6:-1])
            
            if image_size <= 0:
                print("Invalid image data size")
                return
            
            image_data = b''
            while len(image_data) < image_size:
                remaining = image_size - len(image_data)
                packet = client_socket.recv(remaining)
                if not packet:
                    print("Reception fof image data interrupted")
                    return
                image_data += packet
            
            if len(image_data) != image_size:
                print("The received image data is incomplete")
                return
            
            image_array = np.frombuffer(image_data, dtype=np.uint8)
            img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
            
            if img is not None:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
                plt.imshow(img_rgb)
                plt.axis('off') 
                plt.show()

                save_result = cv2.imwrite('received_image.jpg', img)
                if save_result:
                    print("Save image success")
                else:
                    print("Save image failure")
            else:
                print("Can not decode image")
        else:
            print("Error:", header.decode().strip()) 
    except Exception as e:
        print(f"Error: {e}")
    finally:
        client_socket.close()

receive_and_show_image()

### Ask image and save.
- Just like the previous image request, write the image to local.

In [ ]:
import socket

def send_command_to_server():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    try:
        client_socket.connect(('127.0.0.1', 12345))
        # send script to server, can be single command or multiple commands
        
        command = b'Send "^!t"\nSleep 2000\nSend "ls {Enter}"\nSleep 1000\nSend "clear"\nSend "!{F4}"\nSend "#1"\nSleep 4000\nSend "!{F4}"'
        client_socket.sendall(command)
        
    except Exception as e:
        print(f"Error: {e}")
    finally:
        client_socket.close()

send_command_to_server()

### Send script command
See the documentation feature.md for more information about command usage.
- command can be single command or multiple commands
- every command splits with \n.